# 解析環境を構築する

実験を行う解析環境を構築するタスクです。  <br>
本ノートブックで提供している手法は下記です。<br>

 1. NII-RCOSが提供している解析基盤を利用した本環境を解析環境とする
 2. OCS（学認クラウドオンデマンド構築サービス） [注1] を利用した解析環境の構築する

このうち、2.の手法では、ユーザ様に用意いただいたクラウド環境へと解析環境を構築する手法となりますので、<br>
解析環境を利用するための準備をお願いいたします。<br>
OCS/VCPを利用するためのクラウド環境の対象は[リンク](https://nii-gakunin-cloud.github.io/ocs-docs/VCPSDK-doc/#provider)の通りです。<br>

注1: https://cloud.gakunin.jp/ocs/<br>

NII-RCOSが提供している解析基盤を利用した解析環境を新規に構築する場合は、GRDMの解析アドオン機能をご利用ください。

## OCS（学認クラウドオンデマンド）についての補記

OCS（学認クラウドオンデマンド構築サービス）では以下の構成にて、解析環境の構築を行います。<br>

![説明図](./images/RF002003_ocs-figure_01.png)

SaaSとして提供されているOCSを利用することで手法2.を実現いたします。<br>
そのため、OCS利用に伴った利用申請などを行っていただく必要がございます。<br>

## 解析環境を構築する手法を選択する
以下より環境構築を行う手法を選択してください。
 1. 解析基盤を利用した本環境を解析環境とする
 2. OCSで解析環境を構築する
 
※「1.解析基盤を利用した本環境を解析環境とする」を選択した場合は以降の処理は不要となります。研究準備サブフローメニューに戻り次のタスクを実行してください。

In [ ]:
# 環境構築を行う手法を選択する
import os
import traceback
import panel as pn

from library.utils.config import message as msg_config
from library.utils.setting import AnalysisEnvironment
from library.task_director import TaskDirector
from library.utils.widgets import MessageBox
from library.utils.config import path_config, message as msg_config
from library.utils.html.button import create_button
from library.utils.error import InputWarning, UnauthorizedError, NotFoundURLError, UnusableVault
from requests.exceptions import RequestException

notebook_name = 'build_experiment_environment.ipynb'

class ExperimentEnvBuilder(TaskDirector):

    def __init__(self, working_path:str) -> None:
        """ExperimentEnvBuilder コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900

        # 可変フォームボックス
        self._template_form_box = pn.WidgetBox()
        self._template_form_box.width = 900

        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

        # サブフローメニューを表示するボタン
        self.sub_flow_button = self.get_subflow_menu_button_object()

    def set_analysis_env_selector(self):
        """セレクトボックスの設定"""
        self._form_box.clear()

        self.analysis_env = AnalysisEnvironment()
        options = self.analysis_env.get_names()

        self.analysis_env_select = pn.widgets.Select(
            name=msg_config.get('select_ocs_template', 'select_build_tech'),
            options=options,
            size=4,
            width=500
        )
        self.analysis_env_select.param.watch(self.analysis_env_select_callback, 'value')
        self._form_box.append(self.analysis_env_select)
        self.analysis_env_select.param.trigger('value')

    def analysis_env_select_callback(self, event):
        """選択した内容を取得する"""
        self.selected = self.analysis_env_select.value
        self.set_template_form()

    def set_template_form(self):
        """選択した内容によってボタン、メッセージの表示を切り替える"""
        self._msg_output.clear()
        self.analysis_env_id = self.analysis_env.get_id(self.selected)
        description = self.analysis_env.get_description(self.selected)

        if self.analysis_env_id == 'E001':
            self._template_form_box.clear()
            message = pn.pane.Markdown(description)
            self._template_form_box.extend(pn.Column(message, self.sub_flow_button))
            self._form_box.append(self._template_form_box)
        else:
            self._template_form_box.clear()
            md = pn.pane.Markdown(description)
            self._template_form_box.extend(
                pn.Column( md)
            )
            self._form_box.append(self._template_form_box)

    @TaskDirector.task_cell('1')
    def generateFormSection(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # フォーム定義
            self.set_analysis_env_selector()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self.msg_output.update_error(message)
            self.log.error(str(e))
        except InputWarning as e:
            self.msg_output.update_warning(str(e))
            self.log.warning(str(e))
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(message)
            self.log.error(str(e))
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)

ExperimentEnvBuilder(working_path=os.path.abspath('__file__')).generateFormSection()

## OCSテンプレートの取得をする
OCSのテンプレートのフォルダとファイルを ResearchFlow内に展開します。

In [ ]:
# OCSテンプレートの取得をする
import os

import sys
import shutil

import git
from IPython.display import display

sys.path.append('../../../../..')
from library.task_director import TaskDirector

notebook_name = 'build_experiment_environment.ipynb'

class OCSGetter(TaskDirector):

    def __init__(self, working_path: str) -> None:
        """ExperimentEnvCreator コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

    @TaskDirector.task_cell('2')
    def get_template(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()
        local_path = 'ocs-templates'
        if os.path.exists(local_path):
            return
        git.Repo.clone_from(
            url='https://github.com/NII-DG/ocs-templates-dg.git',
            to_path=local_path,
            multi_options=['-b feature/build_env', '--depth 1']
        )
        rm_path = os.path.join(local_path,'.git')
        shutil.rmtree(rm_path)

OCSGetter(working_path=os.path.abspath('__file__')).get_template()

## 解析環境の構築手順を選択する
構築する解析環境を選択します。<br>
リサーチフローで提供している解析環境構築手順は以下となります。
1. The Littlest JupyterHub
2. The Littlest JupyterHub (GPU利用)
3. Open OnDemand
4. JupyterHub
5. Sapporo

構築手順を選択後、ボタンを押下して遷移先の手順に従い構築を行ってください。<br>
<br>
解析環境の構築完了後、本ノートブックへ戻り、以降のタスクを継続してください。<br>


In [ ]:
# 解析環境の構築手順を選択する
import os
import sys
import traceback

import panel as pn
from IPython.display import display

sys.path.append('../../../../..')
from library.task_director import TaskDirector
from library.utils.widgets import MessageBox
from library.utils.config import path_config, message as msg_config
from library.utils.setting import OCSTemplate
from library.utils.html.button import create_button

notebook_name = "build_experiment_environment.ipynb"

class ExperimentEnvBuilder(TaskDirector):

    def __init__(self, working_path:str) -> None:
        """ExperimentEnvBuilder コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900

        # 可変フォームボックス
        self._template_form_box = pn.WidgetBox()
        self._template_form_box.width = 900

        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def set_ocs_template_selector(self):
        """セレクトボックスの設定"""
        self._form_box.clear()
        self._template_form_box.clear()

        self.ocs_template = OCSTemplate()
        options = self.ocs_template.get_name()

        self.ocs_template_list = pn.widgets.Select(
                name=msg_config.get('select_ocs_template', 'ocs_template_title'),
                options=options,
                disabled_options=self.ocs_template.get_disabled_ids(),
                size=4,
                width=500
            )

        self.ocs_template_list.param.watch(self._ocs_template_select_callback, 'value')
        self._form_box.append(self.ocs_template_list)
        self.ocs_template_list.param.trigger('value')

    def _ocs_template_select_callback(self, event):
        """選択した内容を取得する"""
        self.selected = self.ocs_template_list.value
        self.set_templatelink_form()

    def set_templatelink_form(self):
        """選択した内容によってボタン押下後の遷移先を変える"""
        self._msg_output.clear()
        self._template_form_box.clear()

        self.construction_procedure_id = self.ocs_template.get_id(self.selected)
        self.template_path = self.ocs_template.get_template_path(self.selected)
        link_button = self.set_link_button()
        if self.template_path is None:
            raise Exception('Don\'t Get Path of OCS Template')

        if self.construction_procedure_id == "T001":
            # The Littlest JupyterHub
            self._form_box.append(link_button)

        elif self.construction_procedure_id == "T002":
            # The Littlest JupyterHub with GPU
            self._form_box.append(link_button)

        elif self.construction_procedure_id == "T003":
            # Open Ondemand
            self._form_box.append(link_button)

        elif self.construction_procedure_id == "T004":
            # Jupyter Hub
            self._form_box.append(link_button)

        else:
            # Sapporo
            self._form_box.append(link_button)

    #########################
    #  ocs-template link    #
    #########################
    def get_ocs_template_button_object(self)-> pn.pane.HTML:
        """OCS-Templateへのボタンpanel.HTMLオブジェクトの取得
        Returns:
            [panel.pane.HTML]: [HTMLオブジェクト]
        """
        button_width = 500
        ocs_template_link_button = pn.pane.HTML()
        ocs_template_link_button.object = create_button(
            url=self.template_link,
            msg=msg_config.get('select_ocs_template', 'go_template_link'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        ocs_template_link_button.width = button_width
        return ocs_template_link_button

    def set_link_button(self):
        """遷移先を設定する"""
        self.template_link = path_config.get_ocs_template_dir() + self.template_path

        self._template_form_box.extend(
            pn.Column(self.get_ocs_template_button_object())
        )

        return self._template_form_box

    @TaskDirector.task_cell("3")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # フォーム定義
            self.set_ocs_template_selector()

        except UnusableVault:
            message = msg_config.get('form', 'no_vault')
            self.msg_output.update_error(message)
            self.log.error(message)
        except InputWarning as e:
            self.msg_output.update_warning(str(e))
            self.log.warning(str(e))
        except RequestException:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)

ExperimentEnvBuilder(working_path=os.path.abspath('__file__')).generateFormScetion()

## 構築手順を保存する
構築手順をGakunin RDMへ同期するフォルダへと配置します。<br>
構築手順を保存する場合は、以下のコードセルを実行してください。<br>

In [ ]:
# 構築手順を保存する
import os

import sys
from datetime import datetime

from IPython.display import display

sys.path.append('../../../../..')
from library.task_director import TaskDirector
from library.utils.file import copy_dir

notebook_name = 'build_experiment_environment.ipynb'

class OCSGetter(TaskDirector):

    def __init__(self, working_path: str) -> None:
        """ExperimentEnvCreator コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

    @TaskDirector.task_cell('4')
    def get_template(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()
        save_dir_name = datetime.now().strftime("%Y%m%d%H%M%S")
        src = 'ocs-templates/TheLittlestJupyterHub/'
        dst = f'~/data/plan/build_experiment_environment/${save_dir_name}/'
        copy_dir(src, dst, overwrite=True)

OCSGetter(working_path=os.path.abspath('__file__')).get_template()

## Gakunin RDMに保存する
構築手順をGakunin RDMに保存します。<br>

In [ ]:
# Gakunin RDMに実行結果を同期
import os
import sys

import panel as pn
from IPython.display import display

sys.path.append('../../../../..')
from library.task_director import TaskDirector
from library.utils.config import path_config

script_file_name = "build_experiment_environment"
notebook_name = script_file_name+'.ipynb'

class ExperimentEnvBuilder(TaskDirector):

    def __init__(self, working_path:str) -> None:
        """ExperimentEnvBuilder コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

    @TaskDirector.task_cell("5")
    def completed_task(self):
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()

        # フォーム定義
        save_target_path = os.path.join( self._abs_root_path, path_config.DATA, path_config.PLAN, "build_experiment_environment")
        source = self.get_sync_source(save_target_path)
        self.define_save_form(source)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)

    def get_sync_source(self, search_directory :str):
        source = []
        for root, dirs, files in os.walk(search_directory):
            for filename in files:
                if filename.endswith(".ipynb"):
                    path = os.path.join(root, filename)
                    source.append(path)
        return source

ExperimentEnvBuilder(working_path=os.path.abspath('__file__')).completed_task()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..')
from library.task_director import TaskDirector

script_file_name = "build_experiment_environment"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()